In [1]:
%matplotlib qt
import time
import sys
import numpy as np
import pandas as pd
import MySQLdb
import matplotlib.pyplot as plt
sys.path.insert(0, '/Users/kenleejr92/energy_market_project/scripts/MySQL_scripts')
from Query_ERCOT_DB import Query_ERCOT_DB
import cPickle as pickle
from datetime import datetime
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.mllib.linalg import Matrix, Matrices
import pyspark.mllib.linalg.distributed as pydist
from pyspark.sql import SQLContext
from pyspark.mllib.stat import Statistics
from sklearn.cluster import AffinityPropagation
import mpld3
import re
from sets import Set
mpld3.enable_notebook()

class LMP_Query(Query_ERCOT_DB):

    def __init__(self):
        self.SPPs = ['HB_BUSAVG', 
                     'HB_HOUSTON', 
                     'HB_HUBAVG', 
                     'HB_NORTH', 
                     'HB_SOUTH', 
                     'HB_WEST', 
                     'LZ_AEN', 
                     'LZ_CPS', 
                     'LZ_HOUSTON', 
                     'LZ_LCRA', 
                     'LZ_NORTH', 
                     'LZ_RAYBN', 
                     'LZ_SOUTH',  
                     'LZ_WEST']   
        self.node_dict = {}
        self.table_list = []
        self.df = None
        self.CRR_nodes = None
        self.table_boundaries = {'table0':('0001', 'BLUEMD1_8X'),
                                 'table1':('BLUEMD1_8Z', 'CHT_M'),
                                 'table2':('CHT_X', 'DUKE_8405'),
                                 'table3':('DUKE_8505', 'ELEVEE_E8'),
                                 'table4':('ELEVEE_W8', 'GREENLK_L_A'),
                                 'table5':('GREENLK_L_B', 'KEETER'),
                                 'table6':('KEITH', 'L_CEDAHI8_1Y'),
                                 'table7':('L_CEDAHI8_1Z', 'MOSES_1G'),
                                 'table8':('MOSES_2G', 'PHR_8135'),
                                 'table9':('PHR_8140', 'SANDOW1_8Y'),
                                 'table10':('SANDOW_4G', 'TCN7225_BUS'),
                                 'table11': ('TCN7230_BUS', 'VENSW_1777'),
                                 'table12':('VENSW_1785', '_WC_V_C')
                                 }
        for i in range(0,13):
            Query_ERCOT_DB.c.execute("""SHOW columns FROM DAM_LMP%s""" % i)
            result = [r[0] for r in Query_ERCOT_DB.c.fetchall()[2:]]
            self.table_list.append(result)
            for node in result:
                self.node_dict[node] = i
        
    
    def get_CRR_nodes(self):
        Query_ERCOT_DB.c.execute("""SELECT DISTINCT Sink FROM crr_ownership ORDER BY Sink""")
        nodes = list(Query_ERCOT_DB.c.fetchall())
        CRR_prefixes = [r[0] for r in nodes]
        pattern = re.compile('.*_')
        matching_patterns = Set()
        for idx, node in enumerate(CRR_prefixes):
            matches = re.findall(pattern, node)
            if matches: matching_patterns.add(matches[0][:-1])
        # flatten list of lists      
        all_nodes = [item for sublist in self.table_list for item in sublist] + self.SPPs
        self.CRR_nodes = []
        for pattern in matching_patterns:
            pattern2 = re.compile('(.*%s.*)' % pattern)
            for node in all_nodes:
                if re.search(pattern2, node):
                    self.CRR_nodes.append(node)
        return self.CRR_nodes
    
    def query_single_node(self, node):
        s="""SELECT delivery_date, hour_ending, %s from DAM_LMP%s order by delivery_date, hour_ending""" % (node, self.node_dict[node])
        Query_ERCOT_DB.c.execute(s)
        result = list(Query_ERCOT_DB.c.fetchall())
        fresult = []
        for r in result:
            temp = ()
            date = r[0]
            time = str(int(r[1].split(":")[0])-1)
            dt = datetime.strptime(date + " " + time, "%Y-%m-%d %H")
            for x in r[2:]:
                if x == None: x = 0
                temp = temp + (float(x),)
            r = (dt,) + temp
            fresult.append(r)
        self.df = pd.DataFrame(data=[f[1:] for f in fresult], index=[r[0] for r in fresult], columns=[node])
        
    def query(self, nodes, start_date='2011-01-01', end_date='2015-12-31'):
        node_string=''
        for node in nodes:
            node_string = node_string + node + ',' + ' '
        node_string = node_string[:-2]
        s = """SELECT DAM_LMP0.delivery_date, DAM_LMP0.hour_ending, %s 
                                    from DAM_LMP0 join DAM_LMP1 on (DAM_LMP0.delivery_date = DAM_LMP1.delivery_date and DAM_LMP0.hour_ending = DAM_LMP1.hour_ending) 
                                    join DAM_LMP2 on (DAM_LMP0.delivery_date = DAM_LMP2.delivery_date and DAM_LMP0.hour_ending = DAM_LMP2.hour_ending)
                                    join DAM_LMP3 on (DAM_LMP0.delivery_date = DAM_LMP3.delivery_date and DAM_LMP0.hour_ending = DAM_LMP3.hour_ending)
                                    join DAM_LMP4 on (DAM_LMP0.delivery_date = DAM_LMP4.delivery_date and DAM_LMP0.hour_ending = DAM_LMP4.hour_ending)
                                    join DAM_LMP5 on (DAM_LMP0.delivery_date = DAM_LMP5.delivery_date and DAM_LMP0.hour_ending = DAM_LMP5.hour_ending)
                                    join DAM_LMP6 on (DAM_LMP0.delivery_date = DAM_LMP6.delivery_date and DAM_LMP0.hour_ending = DAM_LMP6.hour_ending)
                                    join DAM_LMP7 on (DAM_LMP0.delivery_date = DAM_LMP7.delivery_date and DAM_LMP0.hour_ending = DAM_LMP7.hour_ending)
                                    join DAM_LMP8 on (DAM_LMP0.delivery_date = DAM_LMP8.delivery_date and DAM_LMP0.hour_ending = DAM_LMP8.hour_ending)
                                    join DAM_LMP9 on (DAM_LMP0.delivery_date = DAM_LMP9.delivery_date and DAM_LMP0.hour_ending = DAM_LMP9.hour_ending)
                                    join DAM_LMP10 on (DAM_LMP0.delivery_date = DAM_LMP10.delivery_date and DAM_LMP0.hour_ending = DAM_LMP10.hour_ending)
                                    join DAM_LMP11 on (DAM_LMP0.delivery_date = DAM_LMP11.delivery_date and DAM_LMP0.hour_ending = DAM_LMP11.hour_ending)
                                    join DAM_LMP12 on (DAM_LMP0.delivery_date = DAM_LMP12.delivery_date and DAM_LMP0.hour_ending = DAM_LMP12.hour_ending)
                                    join DAM_SPPs on (DAM_LMP0.delivery_date = DAM_SPPs.delivery_date and DAM_LMP0.hour_ending = DAM_SPPs.hour_ending)
                                    where DAM_LMP0.delivery_date > "%s" and DAM_LMP0.delivery_date < "%s" order by DAM_LMP0.delivery_date, DAM_LMP0.hour_ending;""" % (node_string, start_date, end_date)
        Query_ERCOT_DB.c.execute(s)
        result = list(Query_ERCOT_DB.c.fetchall())
        fresult = []
        for r in result:
            temp = ()
            date = r[0]
            time = str(int(r[1].split(":")[0])-1)
            dt = datetime.strptime(date + " " + time, "%Y-%m-%d %H")
            for x in r[2:]:
                if x == None: x = 0
                temp = temp + (float(x),)
            r = (dt,) + temp
            fresult.append(r)
        self.df = pd.DataFrame(data=[f[1:] for f in fresult], index=[r[0] for r in fresult], columns=nodes)
        return self.df
        
    def get_price(self, node, date, hour_ending):
        for i in range(0,13):
            node = append_n(node)
            if node in self.table_columns['table%s' % i]:
                Query_ERCOT_DB.c.execute("""SELECT %s FROM DAM_LMP%s WHERE delivery_date = "%s" AND hour_ending = \"%s\"""" % (node, i, date, hour_ending))
                result = list(Query_ERCOT_DB.c.fetchall())[0][0]
                return result
        

def append_n(name):
    if name[0] in ['0','1','2','3','4','5','6','7','8','9'] or name == 'LOAD':
        name = 'n' + name
    return name

def dist(x,y):
    cost = 0
    for i in range(len(x)):
        if x[i] == 0 or y[i] == 0:
            continue
        else:
            cost = cost + np.abs(y[i] - x[i])[0]
    return cost
    

/Users/kenleejr92/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: DeprecationWarning: the sets module is deprecated


In [2]:
conf = SparkConf().setMaster("local").setAppName("spark_DAM_correlation")
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)

In [27]:
qcrr = LMP_Query()

In [3]:
crr_nodes = qcrr.get_CRR_nodes()
f = open('crr_nodes.pkl', 'w+')
pickle.dump(crr_nodes, f)
f.close()

In [21]:
np.random.seed(1111)
f2 = open('crr_nodes.pkl', 'r')
crr_nodes = pickle.load(f2)
f2.close()

In [11]:
x1 = qcrr.query(crr_nodes[0:500],'2011-01-01','2015-12-31').as_matrix()
x2 = qcrr.query(crr_nodes[500:1000], '2011-01-01','2015-12-31').as_matrix()
x3 = qcrr.query(crr_nodes[1000:1500], '2011-01-01','2015-12-31').as_matrix()
x4 = qcrr.query(crr_nodes[1500:2000], '2011-01-01','2015-12-31').as_matrix()
x5 = qcrr.query(crr_nodes[2000:2500], '2011-01-01','2015-12-31').as_matrix()
x6 = qcrr.query(crr_nodes[2500:3000], '2011-01-01','2015-12-31').as_matrix()
x7 = qcrr.query(crr_nodes[3000:], '2011-01-01','2015-12-31').as_matrix()
x8 = np.concatenate((x1,x2), axis=1)
x8 = np.concatenate((x8,x3), axis=1)
x8 = np.concatenate((x8,x4), axis=1)
x8 = np.concatenate((x8,x5), axis=1)
x8 = np.concatenate((x8,x6), axis=1)
x8 = np.concatenate((x8,x7), axis=1)

In [16]:
f1 = open('crr_node_prices.csv', 'w+')
np.savetxt('crr_node_prices.csv', x8, delimiter=',', fmt='%10.5f')
f1.close()

In [15]:
x_df = pd.DataFrame.from_csv('/Users/kenleejr92/Downloads/part-0000', header=None, index_col=None)
x = x_df.as_matrix()
for i, item in enumerate(x[:,0]):
    x[i,0] = float(item[1:])
for i, item in enumerate(x[:,3587]):
    x[i,3587] = float(item[:-1])
    

In [33]:
correlation_matrix = np.abs(x)
af = AffinityPropagation(affinity='precomputed').fit(correlation_matrix)

In [49]:
col_dict = {}
for i, col in enumerate(crr_nodes):
    col_dict[i] = col

array([  29,   34,   57,   61,   62,   64,   72,   87,   92,  190,  192,
        208,  210,  213,  216,  222,  259,  280,  290,  297,  326,  327,
        335,  344,  348,  356,  365,  392,  404,  407,  408,  431,  446,
        492,  495,  535,  550,  594,  601,  602,  607,  612,  626,  631,
        633,  635,  637,  651,  659,  663,  669,  674,  683,  690,  747,
        768,  773,  808,  829,  831,  849,  878,  895,  896,  897,  900,
        902,  903,  904,  905,  906,  908,  909,  911,  927,  944,  966,
        971,  979,  990,  991,  997,  998, 1037, 1082, 1091, 1119, 1156,
       1159, 1160, 1186, 1263, 1264, 1265, 1300, 1329, 1345, 1358, 1361,
       1366, 1375, 1407, 1411, 1445, 1452, 1453, 1454, 1457, 1458, 1459,
       1460, 1461, 1462, 1481, 1484, 1505, 1523, 1546, 1555, 1560, 1571,
       1576, 1592, 1608, 1615, 1620, 1680, 1682, 1691, 1716, 1727, 1732,
       1733, 1736, 1737, 1738, 1739, 1740, 1741, 1757, 1777, 1789, 1798,
       1816, 1833, 1834, 1835, 1836, 1839, 1844, 18

In [84]:
cluster_list = [[] for i in range(af.cluster_centers_indices_.size)]
for idx, item in enumerate(af.labels_):
    cluster_list[item].append(col_dict[idx])
exemplars = []
for idx, cluster in enumerate(cluster_list):
    if len(cluster)>10:
        exemplars.append(col_dict[idx])

In [85]:
df = qcrr.query(exemplars)
ax = df.plot()
ax.legend_.remove()
plt.show()

In [82]:
from statsmodels.tsa.seasonal import seasonal_decompose
for i in df.columns:
    x = df[i]
    obj = seasonal_decompose(df[i], freq=60)
    no_hourly = x - obj.seasonal
    obj2 = seasonal_decompose(no_hourly, freq=24*7)
    no_weekly = no_hourly - obj2.seasonal
    plt.plot(no_weekly)
plt.show()

In [62]:
exemplar_matrix = df.as_matrix()
n = len(exemplars)
congestion_matrix = []
for row in exemplar_matrix:
    x = np.zeros((n,n))
    for idx,i in enumerate(row):
        for idx2,j in enumerate(row):
            x[idx,idx2] = i - j
    congestion_matrix.append(x)

TypeError: list indices must be integers, not tuple

In [98]:
from scipy.fftpack import fft
# Number of sample points
y = df.as_matrix()
N = y.shape[0]
# sample spacing
T = 3600.0
x = np.linspace(0.0, N*T, N)
yf = np.fft.fftn(y)
xf = np.linspace(0.0, 1.0/(2.0*T), N/2)
plt.plot(xf, 2.0/N * np.abs(yf[0:N/2]))
plt.grid()
plt.show()

In [50]:
for cluster_num in range(len(cluster_list)):
    cluster_nodes = cluster_list[cluster_num]
    if len(cluster_nodes) >= 4:
        cluster = qcrr.df[cluster_nodes]
        ax = cluster.plot()
        ax.legend()
        plt.show()